### BMIN 5200 Foundations of Artificial Intelligence in Health

### Final Project: Expert System in Epilepsy

Isabella Turco

In [173]:
import clips
import sys
import logging 
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

logging.basicConfig(level=10, format='%(message)s')

# create the CLIPS environment 
env = clips.Environment()

router = clips.LoggingRouter()
env.add_router(router)

## Expert System for Temporal Lobe Epilepsy Lateralization

### Templates (deftemplate) - form that facts can take

In [174]:
# glucest values 
DEFTEMPLATE_GLUCEST_VALUES = """
(deftemplate glucest_values
    (slot hippocampus_right (type FLOAT))
    (slot hippocampus_left (type FLOAT))
    (slot ca1_right (type FLOAT))
    (slot ca1_left (type FLOAT))
    (slot dentategyrus_right (type FLOAT))
    (slot dentategyrus_left (type FLOAT))
    (slot subiculum_right (type FLOAT))
    (slot subiculum_left (type FLOAT))
)
"""
env.build(DEFTEMPLATE_GLUCEST_VALUES)

# asymmetry indices 
DEFTEMPLATE_GLUCEST_ASYMMETRY_VALUES = """
(deftemplate glucest_asymmetry_values
    (slot hippocampus (type FLOAT))
    (slot ca1 (type FLOAT))
    (slot dentategyrus (type FLOAT))
    (slot subiculum (type FLOAT))
)
"""
env.build(DEFTEMPLATE_GLUCEST_ASYMMETRY_VALUES)

DEFTEMPLATE_THRESHOLD = """ 
(deftemplate threshold
    (slot threshold_value (type FLOAT))
)
"""
env.build(DEFTEMPLATE_THRESHOLD)

# lateralization 
DEFTEMPLATE_GLUCEST_LATERALIZATION = """
(deftemplate glucest_lateralization
    (slot hippocampus (type SYMBOL)
        (allowed-symbols right left unclear))
    (slot ca1 (type SYMBOL)
        (allowed-symbols right left unclear))
    (slot dentate_gyrus (type SYMBOL)
        (allowed-symbols right left unclear))
    (slot subiculum (type SYMBOL)
        (allowed-symbols right left unclear))       
    )
"""
env.build(DEFTEMPLATE_GLUCEST_LATERALIZATION)

DEFTEMPLATE_LATERALIZATION_DIAGNOSIS = """
(deftemplate lateralization_diagnosis
    (slot diagnosis (type SYMBOL)
        (allowed-symbols right left mixed-right mixed-left bilateral unclear))
)
"""
env.build(DEFTEMPLATE_LATERALIZATION_DIAGNOSIS)

### Initialize Knowledge Base (deffacts) - create facts

In [175]:
# Add deffacts that the ultimate outputs are all unknown 
DEFFACTS_INITIAL_DIAGNOSIS = """
(deffacts initial_diagnosis_status "Set the initial diagnosis to unknown"
    (lateralization_diagnosis (diagnosis unclear))
)
"""
env.build(DEFFACTS_INITIAL_DIAGNOSIS)

DEFFACTS_THRESHOLD_DEFAULT = """
(deffacts threshold_default "Set the threshold to the default value of 0.25"
    (threshold (threshold_value 0.25))
)
"""
env.build(DEFFACTS_THRESHOLD_DEFAULT)

# reset the environment to make sure the deffacts are added
env.reset()

### Define Rules (defrule) - add facts to knowledge base if they meet certain criteria

In [176]:
# calculate asymmetry indices by subtracting left from right values 
DEFRULE_CALCULATE_GLUCEST_ASYMMETRY = """
(defrule calculate_glucest_asymmetry
    (glucest_values 
        (hippocampus_right ?hc_r) 
        (hippocampus_left ?hc_l)
        (ca1_right ?ca1_r) 
        (ca1_left ?ca1_l)
        (dentategyrus_right ?dg_r) 
        (dentategyrus_left ?dg_l)
        (subiculum_right ?sub_r) 
        (subiculum_left ?sub_l))
=>
    (assert (glucest_asymmetry_values 
        (hippocampus (- ?hc_r ?hc_l))
        (ca1 (- ?ca1_r ?ca1_l))
        (dentategyrus (- ?dg_r ?dg_l))
        (subiculum (- ?sub_r ?sub_l))
    ))
)
"""
env.build(DEFRULE_CALCULATE_GLUCEST_ASYMMETRY)

# determine if each asymmetry index indicates right or left seizure onset zonet 
DEFRULE_CONVERT_ASYMMETRY_TO_LATERALITY = """
(defrule convert_asymmetry_to_laterality
    (glucest_asymmetry_values 
        (hippocampus ?hc_asym)
        (ca1 ?ca1_asym)
        (dentategyrus ?dg_asym)
        (subiculum ?sub_asym))
    (threshold
        (threshold_value ?thr))
=>
    (assert (glucest_lateralization
        (hippocampus (if (> ?hc_asym ?thr) then right else 
                     (if (< ?hc_asym (- 0 ?thr)) then left else unclear)))
        (ca1 (if (> ?ca1_asym ?thr) then right else 
              (if (< ?ca1_asym (- 0 ?thr)) then left else unclear)))
        (dentate_gyrus (if (> ?dg_asym ?thr) then right else 
                        (if (< ?dg_asym (- 0 ?thr)) then left else unclear)))
        (subiculum (if (> ?sub_asym ?thr) then right else 
                    (if (< ?sub_asym (- 0 ?thr)) then left else unclear)))
    ))
)
"""
env.build(DEFRULE_CONVERT_ASYMMETRY_TO_LATERALITY)

In [177]:
DEFRULE_REPORT_LATERALITY_RIGHT = """
(defrule report_laterality_right
    (glucest_lateralization (hippocampus right))
    (glucest_lateralization (ca1 right))
    (glucest_lateralization (dentate_gyrus right))
    (glucest_lateralization (subiculum right))

    ?f1 <-(lateralization_diagnosis (diagnosis unclear))
    =>
    (modify ?f1 (diagnosis right))

    (println "___________")
    (println "All GluCEST values indicate right lateralization.")
    (println "___________")
)
"""
env.build(DEFRULE_REPORT_LATERALITY_RIGHT)

DEFRULE_REPORT_LATERALITY_LEFT = """
(defrule report_laterality_left
    (glucest_lateralization (hippocampus left))
    (glucest_lateralization (ca1 left))
    (glucest_lateralization (dentate_gyrus left))
    (glucest_lateralization (subiculum left))

    ?f1 <-(lateralization_diagnosis (diagnosis unclear))
    =>
    (modify ?f1 (diagnosis left))

    (println "___________")
    (println "All GluCEST values indicate left lateralization.")
    (println "___________")
)
"""
env.build(DEFRULE_REPORT_LATERALITY_LEFT)

DEFRULE_REPORT_LATERALITY_MIXED_RIGHT = """
(defrule report_laterality_mixed_right
    (glucest_lateralization (hippocampus right))

    (glucest_lateralization (ca1 ?ca1_lateralization))
    (glucest_lateralization (dentate_gyrus ?dg_lateralization))
    (glucest_lateralization (subiculum ?sub_lateralization))

    (test (or (neq ?ca1_lateralization ?dg_lateralization)
              (neq ?ca1_lateralization ?sub_lateralization)
              (neq ?dg_lateralization ?sub_lateralization)))

    ?f1 <-(lateralization_diagnosis (diagnosis unclear))
    =>
    (modify ?f1 (diagnosis mixed-right))

    (println "___________")
    (println"GluCEST values reported mixed lateralization results among the subfields, but overall the hippocampus indicates right lateralization.")
    (println "___________")
)
"""
env.build(DEFRULE_REPORT_LATERALITY_MIXED_RIGHT)

DEFRULE_REPORT_LATERALITY_MIXED_LEFT = """
(defrule report_laterality_mixed_left
    (glucest_lateralization (hippocampus left))

    (glucest_lateralization (ca1 ?ca1_lateralization))
    (glucest_lateralization (dentate_gyrus ?dg_lateralization))
    (glucest_lateralization (subiculum ?sub_lateralization))

    (test (or (neq ?ca1_lateralization ?dg_lateralization)
              (neq ?ca1_lateralization ?sub_lateralization)
              (neq ?dg_lateralization ?sub_lateralization)))

    ?f1 <-(lateralization_diagnosis (diagnosis unclear))
    =>
    (modify ?f1 (diagnosis mixed-left))

    (println "___________")
    (println"GluCEST values reported mixed lateralization results among the subfields, but overall the hippocampus indicates left lateralization.")
    (println "___________")
)
"""
env.build(DEFRULE_REPORT_LATERALITY_MIXED_LEFT)

DEFRULE_REPORT_LATERALITY_BILATERAL = """
(defrule report_laterality_bilateral
    (declare (salience 1))
    (glucest_asymmetry_values 
        (hippocampus ?hc_asym)
        (ca1 ?ca1_asym)
        (dentategyrus ?dg_asym)
        (subiculum ?sub_asym))

    (test (and (>= ?hc_asym -0.05) (<= ?hc_asym 0.05)))
    (test (and (>= ?ca1_asym -0.05) (<= ?ca1_asym 0.05)))
    (test (and (>= ?dg_asym -0.05) (<= ?dg_asym 0.05)))
    (test (and (>= ?sub_asym -0.05) (<= ?sub_asym 0.05)))

    ?f <- (lateralization_diagnosis (diagnosis unclear))
    => 
    (modify ?f (diagnosis bilateral))

    (println "___________")
    (println "All GluCEST values indicate bilateral involvement or no clear lateralization.")
    (println "___________")
)
"""
env.build((DEFRULE_REPORT_LATERALITY_BILATERAL))

DEFRULE_REPORT_LATERALITY_UNCLEAR = """
(defrule report_laterality_unclear
    (declare (salience 0))
    (glucest_lateralization (hippocampus unclear))
    (glucest_lateralization (ca1 unclear))
    (glucest_lateralization (dentate_gyrus unclear))
    (glucest_lateralization (subiculum unclear))
    (not (lateralization_diagnosis (diagnosis bilateral)))

    ; ?f1 <-(lateralization_diagnosis (diagnosis unclear))
    =>
    ; (modify ?f1 (diagnosis unclear))

    (println "___________")
    (println "GluCEST values reported unclear results, potentially indicating bilaterality or more complex pathology. Refer patient for more testing.")
    (println "___________")
)
"""
env.build(DEFRULE_REPORT_LATERALITY_UNCLEAR)

### Execution

#### Example 1: Patient with Right Temporal Lobe Epilepsy

In [178]:
env.reset();
# load gluCEST values (units: percentage contrast) extracted from a 3D gluCEST scan
env.assert_string("""
    (glucest_values 
        (hippocampus_right 7.15)
        (hippocampus_left 6.57)
        (ca1_right 7.12)
        (ca1_left 6.76)
        (dentategyrus_right 7.81)
        (dentategyrus_left 6.48)
        (subiculum_right 6.00)
        (subiculum_left 5.52)
    )
    ; choose any percentage threshold for categorizing asymmetry indices to laterality 
    (threshold
        (threshold_value 0.25)
    )
""")
env.run();

# print_facts(env)

___________
All GluCEST values indicate right lateralization.
___________


#### Example 2: Patient with Mixed Results but Right Temporal Lobe Epilepsy

In [179]:
env.reset();
# load gluCEST values (units: percentage contrast) extracted from a 3D gluCEST scan
env.assert_string("""
    (glucest_values 
        (hippocampus_right 8.92)
        (hippocampus_left 7.75)
        (ca1_right 9.22)
        (ca1_left 7.95)
        (dentategyrus_right 8.41)
        (dentategyrus_left 8.80)
        (subiculum_right 7.04)
        (subiculum_left 7.21)
    )
    ; choose any percentage threshold for categorizing asymmetry indices to laterality 
    (threshold
        (threshold_value 0.25)
    )
""")
env.run();

# print_facts(env)

___________
GluCEST values reported mixed lateralization results among the subfields, but overall the hippocampus indicates right lateralization.
___________


### Example 3: Patient with Unclear Lateralization

In [180]:
env.reset();
# load gluCEST values (units: percentage contrast) extracted from a 3D gluCEST scan
env.assert_string("""
    (glucest_values 
        (hippocampus_right 7.72)
        (hippocampus_left 7.83)
        (ca1_right 7.04)
        (ca1_left 7.13)
        (dentategyrus_right 7.98)
        (dentategyrus_left 8.15)
        (subiculum_right 7.01)
        (subiculum_left 6.89)
    )
    ; choose any percentage threshold for categorizing asymmetry indices to laterality 
    (threshold
        (threshold_value 0.25)
    )
""")
env.run();

# print_facts(env)

___________
GluCEST values reported unclear results, potentially indicating bilaterality or more complex pathology. Refer patient for more testing.
___________


### Example 4: Patient with Bilaterality

In [181]:
env.reset();
# load gluCEST values (units: percentage contrast) extracted from a 3D gluCEST scan
env.assert_string("""
    (glucest_values 
        (hippocampus_right 9.12)
        (hippocampus_left 9.09)
        (ca1_right 8.67)
        (ca1_left 8.68)
        (dentategyrus_right 8.92)
        (dentategyrus_left 8.96)
        (subiculum_right 9.20)
        (subiculum_left 9.16)
    )
    ; choose any percentage threshold for categorizing asymmetry indices to laterality 
    (threshold
        (threshold_value 0.25)
    )
""")
env.run();

# print_facts(env)

___________
All GluCEST values indicate bilateral involvement or no clear lateralization.
___________


## Ablation Analysis

In [182]:
def ablation_analysis_results(env):
    diagnoses = []
    for fact in env.facts():
        if fact.template.name == "lateralization_diagnosis":
            diagnoses.append(fact["diagnosis"])
    
    if not diagnoses:
        print("No lateralization diagnosis found.")
    return diagnoses

In [183]:
# test with example 1 
rules = ["calculate_glucest_asymmetry", "convert_asymmetry_to_laterality", "report_laterality_right", "report_laterality_left", "report_laterality_mixed_right", "report_laterality_mixed_left", "report_laterality_unclear"]
results = {}
for rule in rules:
    env.reset()
    env.eval(f'(undefrule {rule})')
    env.assert_string("""
    (glucest_values 
        (hippocampus_right 7.15)
        (hippocampus_left 6.57)
        (ca1_right 7.12)
        (ca1_left 6.76)
        (dentategyrus_right 7.81)
        (dentategyrus_left 6.48)
        (subiculum_right 6.00)
        (subiculum_left 5.52)
    )
    """)
    env.run()
    results[rule] = ablation_analysis_results(env)

print(results) # it also prints out which rule is deactitvated and therefore unable to be found 

{'calculate_glucest_asymmetry': ['unclear'], 'convert_asymmetry_to_laterality': ['unclear'], 'report_laterality_right': ['unclear'], 'report_laterality_left': ['unclear'], 'report_laterality_mixed_right': ['unclear'], 'report_laterality_mixed_left': ['unclear'], 'report_laterality_unclear': ['unclear']}


In [184]:
# test with example 2: mixed results but right lateralization 
rules = ["calculate_glucest_asymmetry", "convert_asymmetry_to_laterality", "report_laterality_right", "report_laterality_left", "report_laterality_mixed_right", "report_laterality_mixed_left", "report_laterality_unclear"]
results = {}
for rule in rules:
    env.reset()
    env.eval(f'(undefrule {rule})')
    env.assert_string("""
    (glucest_values 
        (hippocampus_right 8.92)
        (hippocampus_left 7.75)
        (ca1_right 9.22)
        (ca1_left 7.95)
        (dentategyrus_right 8.41)
        (dentategyrus_left 8.80)
        (subiculum_right 7.04)
        (subiculum_left 7.21)
    )
    """)
    env.run()
    results[rule] = ablation_analysis_results(env)

print(results) # it also prints out which rule is deactitvated and therefore unable to be found 

[PRNTUTIL1] Unable to find defrule 'calculate_glucest_asymmetry'.
[PRNTUTIL1] Unable to find defrule 'convert_asymmetry_to_laterality'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_right'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_left'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_mixed_right'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_mixed_left'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_unclear'.


{'calculate_glucest_asymmetry': ['unclear'], 'convert_asymmetry_to_laterality': ['unclear'], 'report_laterality_right': ['unclear'], 'report_laterality_left': ['unclear'], 'report_laterality_mixed_right': ['unclear'], 'report_laterality_mixed_left': ['unclear'], 'report_laterality_unclear': ['unclear']}


In [185]:
# test with example 3: unclear lateralization/potentially bilateral temporal lobe epilepsy
rules = ["calculate_glucest_asymmetry", "convert_asymmetry_to_laterality", "report_laterality_right", "report_laterality_left", "report_laterality_mixed_right", "report_laterality_mixed_left", "report_laterality_unclear"]
results = {}
for rule in rules:
    env.reset()
    env.eval(f'(undefrule {rule})')
    env.assert_string("""
    (glucest_values 
        (hippocampus_right 7.15)
        (hippocampus_left 6.57)
        (ca1_right 7.12)
        (ca1_left 6.76)
        (dentategyrus_right 7.81)
        (dentategyrus_left 6.48)
        (subiculum_right 6.00)
        (subiculum_left 5.52)
    )
    """)
    env.run()
    results[rule] = ablation_analysis_results(env)

print(results) # it also prints out which rule is deactitvated and therefore unable to be found 

[PRNTUTIL1] Unable to find defrule 'calculate_glucest_asymmetry'.
[PRNTUTIL1] Unable to find defrule 'convert_asymmetry_to_laterality'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_right'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_left'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_mixed_right'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_mixed_left'.
[PRNTUTIL1] Unable to find defrule 'report_laterality_unclear'.


{'calculate_glucest_asymmetry': ['unclear'], 'convert_asymmetry_to_laterality': ['unclear'], 'report_laterality_right': ['unclear'], 'report_laterality_left': ['unclear'], 'report_laterality_mixed_right': ['unclear'], 'report_laterality_mixed_left': ['unclear'], 'report_laterality_unclear': ['unclear']}
